In [3]:
import pandas as pd

# import Dataset 
cd = pd.read_csv('C:\\Users\\SHRAVANI\\Downloads\\Entertainment.csv', encoding = 'utf8')
cd.shape # shape
cd.columns
cd.Category # genre columns

from sklearn.feature_extraction.text import TfidfVectorizer #term frequencey- inverse document frequncy is a numerical statistic that is intended to reflect how important a word is to document in a collecion or corpus

# Creating a Tfidf Vectorizer to remove all stop words
tfidf = TfidfVectorizer(stop_words = "english")    # taking stop words from tfid vectorizer 

# replacing the NaN values in overview column with empty string
cd["Category"].isnull().sum() 
cd["Category"] = cd["Category"].fillna(" ")

# Preparing the Tfidf matrix by fitting and transforming
tfidf_matrix = tfidf.fit_transform(cd.Category)   #Transform a count matrix to a normalized tf or tf-idf representation
tfidf_matrix.shape #12294, 46

# with the above matrix we need to find the similarity score
# There are several metrics for this such as the euclidean, 
# the Pearson and the cosine similarity scores

# For now we will be using cosine similarity matrix
# A numeric quantity to represent the similarity between 2 movies 
# Cosine similarity - metric is independent of magnitude and easy to calculate 

# cosine(x,y)= (x.y⊺)/(||x||.||y||)

from sklearn.metrics.pairwise import linear_kernel

# Computing the cosine similarity on Tfidf matrix
cosine_sim_matrix = linear_kernel(tfidf_matrix, tfidf_matrix)

# creating a mapping of anime name to index number 
cd_index = pd.Series(cd.index, index = cd['name']).drop_duplicates()

cd_id = cd_index["Toy Story (1995)"]
cd_id

def get_recommendations(name, topN):    
    # topN = 10
    # Getting the movie index using its title 
    cd_id = cd_index[name]
    
    # Getting the pair wise similarity score for all the anime's with that 
    # anime
    cosine_scores = list(enumerate(cosine_sim_matrix[cd_id]))
    
    # Sorting the cosine_similarity scores based on scores 
    cosine_scores = sorted(cosine_scores, key=lambda x:x[1], reverse = True)
    
    # Get the scores of top N most similar movies 
    cosine_scores_N = cosine_scores[0: topN+1]
    
    # Getting the movie index 
    anime_idx  =  [i[0] for i in cosine_scores_N]
    anime_scores =  [i[1] for i in cosine_scores_N]
    
    # Similar movies and scores
    cd_similar_show = pd.DataFrame(columns=["name", "Score"])
    cd_similar_show["name"] = cd.loc[anime_idx, "name"]
    cd_similar_show["Score"] = anime_scores
    cd_similar_show.reset_index(inplace = True)  
    # anime_similar_show.drop(["index"], axis=1, inplace=True)
    print (cd_similar_show)
    # return (anime_similar_show)

    
# Enter your anime and number of anime's to be recommended 
get_recommendations("Sabrina (1995)", topN = 10)
cd_index["Sabrina (1995)"]


    index                                               name     Score
0       6                                     Sabrina (1995)  1.000000
1      23                                      Powder (1995)  0.567625
2      13                                       Nixon (1995)  0.548868
3      19                                 Money Train (1995)  0.534361
4      24                           Leaving Las Vegas (1995)  0.421340
5       1                                     Jumanji (1995)  0.320244
6      42               How to Make an American Quilt (1995)  0.314217
7      15                                      Casino (1995)  0.267914
8      29  Shanghai Triad (Yao a yao yao dao waipo qiao) ...  0.266933
9      22                                   Assassins (1995)  0.250039
10     35                                    Clueless (1995)  0.210673


6